In [17]:
# conda activate dream

library(edgeR)
library(parallelly)
library(data.table)
library(BiocParallel)
library(variancePartition)

setwd("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM")

Sys.setenv(OPENBLAS_NUM_THREADS="1", OMP_NUM_THREADS="1", MKL_NUM_THREADS="1")
if (requireNamespace("RhpcBLASctl", quietly=TRUE)) {
  RhpcBLASctl::blas_set_num_threads(1)
  RhpcBLASctl::omp_set_num_threads(1)
}

param <- SnowParam(workers=6, type="SOCK", progressbar=TRUE)

Loading required package: limma

Loading required package: ggplot2


Attaching package: ‘variancePartition’


The following objects are masked from ‘package:limma’:

    eBayes, topTable




Here I run DE analysis on the pseudobulked cell type data

In [ ]:
counts <- fread("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk.csv", data.table=FALSE)
sample_meta <- fread("data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_sampleinfo.csv", data.table=FALSE)

# Add sex and age info. to metadata
sample_meta$Row <- 1:nrow(sample_meta)
donor_meta <- fread("/mnt/lareaulab/reliscu/projects/NSF_GRFP/data/scRNA-seq/tasic_2018/tasic_2018_tableS10_sampleinfo_donor_level.csv")
sample_meta <- merge(sample_meta, donor_meta[,1:3], by.x="Donor", by.y="Animal ID", all.x=TRUE, sort=FALSE)
sample_meta <- sample_meta[order(sample_meta$Row),] # Keep original row order
rownames(sample_meta) <- sample_meta$Sample_ID

# Reformat cell type names
sample_meta$Cell_type <- sapply(sample_meta$Cell_type, function(x) gsub(" ", "_", x))
sample_meta$Cell_type <- sapply(sample_meta$Cell_type, function(x) gsub("/", "_", x, fixed=TRUE))

# Center age
sample_meta$Age_c <- as.numeric(scale(sample_meta$Age, center=TRUE, scale=FALSE))

# Prep count data

y <- DGEList(counts) 
keep <- filterByExpr(y, group=sample_meta$Cell_type)
y <- y[keep,, keep.lib.sizes=FALSE]
print(dim(y$counts))
y <- calcNormFactors(y)

ctypes <- unique(sample_meta$Cell_type)
ctype_levels <- levels(factor(ctypes))

In [ ]:
# # Subset data for testing

# set.seed(1)
# sample_idx <- sample(1:nrow(sample_meta), size=200)
# sample_meta <- sample_meta[sample_idx,]
# counts_subset <- counts[sample(1:nrow(counts), size=1000), c(1, sample_idx + 1)]

# y <- DGEList(counts_subset)
# keep <- filterByExpr(y, group=sample_meta$Cell_type)
# y <- y[keep,, keep.lib.sizes=FALSE]
# dim(y$counts)
# y <- calcNormFactors(y)

# ctypes <- unique(sample_meta$Cell_type)
# ctype_levels <- levels(factor(ctypes))

# 1 vs. pooled tests

Compare gene expression between target cell type all other cell types pooled

In [ ]:
# Note: this takes a few hours 

form <- ~ Test + Sex + Age_c + (1 | Donor)

pooled_res_list <- lapply(ctypes, function(target) {
    print(paste(target, "vs. rest"))
    sample_meta$Test <- ifelse(
        sample_meta$Cell_type == target, target, "Rest"
    )
    sample_meta$Test <- factor(sample_meta$Test, levels=c("Rest", target))
    vobj <- voomWithDreamWeights(y, form, sample_meta, BPPARAM=param)
    fit <- dream(vobj, form, sample_meta, BPPARAM=param)
    fit <- eBayes(fit)
    topTable(fit, coef=paste0("Test", target), number=Inf, p.value=0.05)
})
names(pooled_res_list) <- ctypes

saveRDS(pooled_res_list, file="data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_pooled_DE_genes_dream.RDS")

# Pairwise tests

Compare model coefficients expression between each target cell type each other cell type (pairwise)

In [9]:
form <- ~ 0 + Cell_type + Sex + Age_c + (1 | Donor)
vobj <- voomWithDreamWeights(y, form, sample_meta, BPPARAM=param)

In [ ]:
form_fixed <- ~ 0 + Cell_type + Sex + Age_c
coef_names <- colnames(model.matrix(form_fixed, data=sample_meta))
ctype_coef_names <- coef_names[-c(length(coef_names)-1, length(coef_names))]
ctype_pairs <- combn(ctype_coef_names, m=2) 

L_list <- lapply(1:ncol(ctype_pairs), function(idx) {
    pair <- ctype_pairs[,idx]
    ctype1 <- gsub("Cell_type", "", pair[1])
    ctype2 <- gsub("Cell_type", "", pair[2])
    L <- matrix(
        0, nrow=length(coef_names), ncol=1, 
        dimnames=list(coef_names, paste0(ctype1, "_vs_", ctype2))
    )
    L[pair[1], 1] <-  1
    L[pair[2], 1] <- -1
    L
})
L <- do.call(cbind, L_list)

In [ ]:
# Batching contrasts

idx_list <- split(seq_len(ncol(L)), ceiling(seq_along(seq_len(ncol(L))) / 100))
res_list <- vector("list", length(idx_list))

for (i in seq_along(idx_list)) {
    print(paste("Starting batch", i))
    idx <- idx_list[[i]]
    Lsub <- L[,idx, drop=FALSE]
    fit  <- dream(vobj, form, sample_meta, L=Lsub, BPPARAM=param)
    # fit <- eBayes(fit)
    batch_res <- lapply(colnames(Lsub), function(test) {
        tt <- topTable(fit, coef=test, number=Inf, p.value=0.05)
        tt$Test <- test
        tt
    })
    res_list[[i]] <- do.call(rbind, batch_res)
    rm(fit); gc()
}
pairwise_res <- do.call(rbind, res_list)
pairwise_res_list <- tapply(pairwise_res, pairwise_res$Test, list)

Process pairwise DE results

In [ ]:
# Each test (e.g. CT1 and CT2 comparison) returns both positive and negative LFC DE genes
# To simplify downstream analyses, I want to split these out, e.g. make separate tables for CT1 vs. CT2 DE genes, and CT2 vs. CT1 DE genes
# This will require subsetting tables from a given comparison to genes with only positive (or negative) LFC values, depending on which cell type was the reference level

pairwise_res_list_split <- lapply(seq_along(pairwise_res_list), function(i) {
    pairwise_res <- pairwise_res_list[[i]]
    # Note: the first cell type in the test will always be the reference level...
    #   due to the fact each cell type was tested against other cell types that had names that came AFTER it (in alphabetical order),
    #   which is how factor levels get assigned
    ctype_res <- lapply(c("<", ">"), function(func_str) {
        func <- match.fun(func_str)
        pairwise_res[func(pairwise_res$logFC, 0),]
    })
    dir1_test <- names(pairwise_res_list)[i]
    ctypes <- unlist(strsplit(dir1_test, "_vs_"))
    test_names <- c(
        dir1_test, 
        paste0(ctypes[2], "_vs_", ctypes[1])
    )
    names(ctype_res) <- test_names
    ctype_res 
})

In [14]:
pairwise_res_split <- unlist(pairwise_res_list_split, recursive=FALSE)

In [ ]:
saveRDS(pairwise_res_split, file="data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_pairwise_DE_genes_dream.RDS")

# 1 vs. mean tests

Compare model coefficients between each target cell type and the average of the other cell types (this approach is invariant to cell type composition, unlike the pooled approach)

In [ ]:
form <- ~ 0 + Cell_type + Sex + Age_c + (1 | Donor)
vobj <- voomWithDreamWeights(y, form, sample_meta, BPPARAM=param)

In [ ]:
# Make a 1 vs. mean(others) contrast matrix

form_fixed <- ~ 0 + Cell_type + Sex + Age_c
coef_names <- colnames(model.matrix(form_fixed, data=sample_meta))

make_mean_contrast <- function(target, ctype_levels){
  others <- setdiff(ctype_levels, target)
  K <- length(others)
  L <- matrix(0, length(coef_names), 1, dimnames=list(coef_names, paste0(target, "_vs_meanOthers")))
  L[paste0("Cell_type", make.names(target)), 1] <-  1
  L[paste0("Cell_type", make.names(others)), 1] <- -1/K
  L
}
L_list <- lapply(ctypes, make_mean_contrast, ctype_levels)
L <- do.call(cbind, L_list)

In [ ]:
fit  <- dream(vobj, form, sample_meta, L=L, BPPARAM=param)
# fit <- eBayes(fit)

In [ ]:
mean_res_list <- lapply(colnames(L), function(test) {
    tt <- topTable(fit, coef=test, number=Inf, p.value=0.05)
    tt$Test <- test
    tt
})
names(mean_res_list) <- ctype_levels

In [ ]:
saveRDS(mean_res_list, file="data/tasic_2018_ALM_STAR_donor_cell_type_pseudobulk_1_vs_meanOthers_DE_genes_dream.RDS")